In [6]:
from officelib.xllib import *
from officelib.const import xlconst as xlc
from hello.hello3 import HelloApp, NotLoggedInError

In [7]:
def call(f, *args, m=None):
    if m is None:
        s = "(" + ", ".join(map(repr, args)) + ")"
        m = "%s%s" % (f.__name__, s)
    print(m)
    while True:
        try:
            rv = f(*args)
        except NotLoggedInError:
            h.login()
        except requests.exceptions.ConnectTimeout as e:
            print("Make sure you're on the right network!")
            raise
        else:
            return rv

In [8]:
def bt():
    import datetime; return datetime.datetime.now().strftime("%y%m%d")

def dl(bn, p):
    if isinstance(bn, str):
        b = call(h.getdatareport_bybatchname, bn)
    elif isinstance(bn, tuple):
        name, d1, d2 = bn
        b = call(h.getreport_bydate, 'process_data', d1, d2)
        bn = name
    fn = p+bn+"15L " + bt() + ".csv"
    with open(fn, 'wb') as f:
        f.write(b)
    return fn

In [30]:
def open_file(file):
    global xl, wb, ws, cells, cr
    global ws2, cells2, cr2
    try:
        xl
    except NameError:
        xl = Excel()
    try:
        xl.Visible
    except Exception:
        xl = Excel()
    wb = xl.Workbooks.Open(file)
    ws2 = wb.Worksheets(1)
    cells2 = ws2.Cells
    cr2 = cells2.Range
    
    ws = wb.Worksheets.Add()
    cells = ws.Cells
    cr = cells.Range

def elform(c, ref):
    a1 = c.GetAddress(0,0)
    a2 = ref.GetAddress(1,1)
    a3 = ref.Offset(0,2).GetAddress(1,1)
    form = "=(%s-%s)*%s"%(a1, a2, a3)
    return form
    
def mk_elapsed(c, ref, ins=True):
    
    if ins:
        c.Offset(1,2).EntireColumn.Insert()
    top = c.Offset(1,2)
    bot = c.End(xlc.xlDown).Offset(1,2)
    top.Value = elform(c, ref)
    r = cr(top, bot)
    top.AutoFill(r)
    r.NumberFormat = "0.00"
    
def chart_rng(c):
    x1 = c.Offset(1,2)
    x2 = x1.End(xlc.xlDown)
    y1 = c.Offset(1,3)
    y2 = y1.End(xlc.xlDown)
    return cr(x1, x2), cr(y1, y2)

def mkch(c,v):
    chart = CreateChart(ws, xlc.xlXYScatterLinesNoMarkers)
    mkser(chart, c,v)
    chart.ApplyLayout(8)
    return chart
    
def mkser(ch, c,v):
    x, y = chart_rng(c)
    return CreateDataSeries(ch, x, y, v)
    
def fixaxes():
    cl = [co.Chart for co in ws.ChartObjects()]
    cl.extend(wb.Charts)
    for c in cl:
        ax = c.Axes(xlc.xlCategory, xlc.xlPrimary)
        ax.MinimumScale = 0
        ax.HasTitle = True
        ax.AxisTitle.Text = "Time (hr)"
        
        ax2 = c.Axes(xlc.xlValue, xlc.xlPrimary)
        ax2.HasTitle = True
        
    for c in wb.Charts:
        ax2 = c.Axes(xlc.xlValue, xlc.xlPrimary)
        ax2.AxisTitle.Text = c.Name
        
    
def analyze(params=(), ref1s=None):
    if not params:
        return
    if ref1s is None:
        ref1s = params[0][0][0]
    c1 = cells.Find(ref1s).Offset(2,1)
    ref1 = c1
    mk_elapsed(c1, ref1)
    ref1.Offset(0, 2).Value = "24"
    
    for (ref, sp), vars in params:
        
        # make chart
        print("Making chart for", ref)
        c = cells.Find(ref).Offset(2,1)
        if ref != ref1s:
            mk_elapsed(c, ref1)
        ch1 = mkch(c, ref)
        ch2 = mkch(c, ref)
        
        # add set point
        c = cells.Find(sp).Offset(2,1)
        mk_elapsed(c, ref1)
        for ch in ch1, ch2:
            s = mkser(ch, c, sp)
            s.MarkerSize = 2
            s.MarkerStyle = 1
            s.Border.ColorIndex = xlc.xlNone
            s.MarkerForegroundColor = 0
            s.MarkerBackgroundColor = 0
        
        for v in vars:
            print("Adding data series", v)
            c = cells.Find(v).Offset(2,1)
            mk_elapsed(c, ref1)
            s=mkser(ch2, c, v)
            s.AxisGroup = xlc.xlSecondary
        ch1.Location(xlc.xlLocationAsNewSheet, ref)
        
    # special for DO
    ch = wb.Charts("DOPV(%)")
    c = cells.Find("DOModeActual")
    mk_elapsed(c, ref1)
    s = mkser(ch, c, "DOModeActual")
    s.AxisGroup = xlc.xlSecondary
    s.ChartType = xlc.xlXYScatter
    s.MarkerSize = 4
    
    # MFCs
    c = cells.Find("MainGasActualRequest(LPM)").Offset(2,1)
    mk_elapsed(c, ref1)
    ch3 = mkch(c, "MainGasActualRequest(LPM)")
    for s in ("Air", "N2", "CO2", "O2"):
        m = "MFC%sFlowFeedback(LPM)"%s
        c = cells.Find(m).Offset(2,1)
        mk_elapsed(c, ref1)
        s=mkser(ch3, c, s)
    ch3.Location(xlc.xlLocationAsNewSheet, "Gases")

    fixaxes()
    wb.SaveAs(fn.replace('.csv','.xlsb'), FileFormat=xlc.xlExcel12, AddToMru=True)
        
def migrate_data():
    for col in list(ws2.UsedRange.Columns)[::3]:
        c1 = col.Cells(1,1)
        c2 = c1.End(xlc.xlDown).Offset(1,2)
        step = (c2.Row - c1.Row) // 30000 + 1
        
        data = cr2(c1, c2).Value
        if step > 1:
            data = data[::step]
            
        c3 = ws.Columns(c1.Column).Cells(1,1)
        c4 = c3.Offset(len(data), 2)
        
        cr(c3, c4).Value = data
        
        

        
def analf(f, params=(), ref1=None):
    open_file(f)
    migrate_data()
    analyze(params, ref1)
    wb.Close()

In [31]:
#h = HelloApp('192.168.1.3')
p='C:\\PBSCloudStation\\(2) R&D-Product Engineering\\Software Development\\3.0 Project\\Phase 2 Working Copy\\IE00005 PID Tuning\\IE00005-08 Bio Lab Test'
#dl("kls-pbs exp9 n-0", p)

In [32]:
import os
fn = os.path.join(p, "15L Biolab Data 170630.csv")

params = [(("DOPV(%)", "DOSP(%)"), ("DOO2FlowControllerRequestLimited(%)", "DON2FlowActualRequest(%)")),
         (("pHPV", "pHSP"), ("pHCO2ActualRequest(%)",)),
         (("TempPV(C)", "TempSP(C)"), ("TempHeatDutyActual(%)",)),
         (("AgPV(RPM)","AgSP(RPM)"), ("AgPowerActualRequest(%)",)),
         ]

analf(fn, params)
xl.Visible=True
xl.RecentFiles(1).Open()

Making chart for DOPV(%)
Adding data series DOO2FlowControllerRequestLimited(%)
Adding data series DON2FlowActualRequest(%)
Making chart for pHPV
Adding data series pHCO2ActualRequest(%)
Making chart for TempPV(C)
Adding data series TempHeatDutyActual(%)
Making chart for AgPV(RPM)
Adding data series AgPowerActualRequest(%)


<win32com.gen_py.None.Workbook>

In [ ]:
from hello.hello3 import HelloApp
from datetime import datetime
%matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import *
from matplotlib import ticker

In [ ]:
def data(span, group, typ="last"):
    h = HelloApp("71.189.82.196:81")
    td = h.getTrendData(span, group)
    x, y = list(zip(*td[typ]))
    x2 = [datetime.fromtimestamp(d/1000) for d in x]
    first = x2[0]
    x3 = [(d-first).total_seconds()/60 for d in x2]
    return x3, y

In [ ]:
def plotxy(ax, x, y):
    ax.plot(x,y)
    ax.yaxis.set_major_formatter(ticker.FormatStrFormatter("%s"))

def plot(span='15min', group='agitation'):
    plt.close()
    x, y = data(span, group)
    fig = plt.figure()
    ax = plt.add_subplot(1,1,1)
    plotxy(ax, x, y)

In [ ]:
plot('24hr')

In [ ]:
def plot2(group='agitation'):
    fig = plt.figure()
    for i, span in enumerate(("15min", "2hr", "12hr"), 1):
        ax = fig.add_subplot(3, 1, i)
        for typ in "min", "max", "last":
            x, y = data(span, group, typ)
            plotxy(ax, x, y)
        ax.set_ylabel(span)

In [ ]:
plot2()

In [ ]:
ax = plt.gca()

In [ ]:
plt.gcf().axes[0].set_ylabel("15 min")
plt.gcf().axes[1].set_ylabel("2 hr")
plt.gcf().axes[2].set_ylabel("12 hr")
for ax in plt.gcf().axes:
    ax.set_xlabel("Time(min)")